In [ ]:
import requests
from flask import Flask,request,jsonify
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from math import ceil
import pickle
import tensorflow as tf
from tensorflow.keras.regularizers import l1
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint, EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from sklearn.datasets import load_files
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

In [ ]:
model = tf.keras.models.load_model("LSTM-TEXT-Classifictaion_V2-03-0.55.hdf5")
with open("X_Data.pkl", "rb") as DS:
    X = pickle.load(DS)
with open("ML_Model.pkl", "rb") as DS:
    modelML = pickle.load(DS)
with open("Word2ID.pkl", "rb") as DS:
    word2ID = pickle.load(DS)

In [ ]:
def remove_emojis(data):
    emoj = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
        u"\U00002500-\U00002BEF"  # chinese char
        u"\U00002702-\U000027B0"
        u"\U00002702-\U000027B0"
        u"\U000024C2-\U0001F251"
        u"\U0001f926-\U0001f937"
        u"\U00010000-\U0010ffff"
        u"\u2640-\u2642" 
        u"\u2600-\u2B55"
        u"\u200d"
        u"\u23cf"
        u"\u23e9"
        u"\u231a"
        u"\ufe0f"  # dingbats
        u"\u3030"
                      "]+", re.UNICODE)
    return re.sub(emoj, '', data)

In [ ]:
def PreProcess(text,word2ID):
    Final_Preprocced_Data = []
    Sent = text
    D = remove_emojis(Sent)
    tempList = []
    SplitedData = re.split(r'[a-z 1 2 3 4 5 6 7 8 9 0 ^ ( ) , ، % $ : ; "  > < ! ? ؟  ; _ -|, |\*|\n . @ ? ! # / \xa0]'
                           ,D) 
    for words in SplitedData:
        if (words != ""):
            tempList.append(words)

    WholeArabicSent = ' '.join(tempList)
    Final_Preprocced_Data.append(WholeArabicSent)
    
    X_Vector = []
    for data in Final_Preprocced_Data:
        Vec = []
        words = nltk.word_tokenize(data)
        for word in words:
            if word in word2ID.keys():
                Id = word2ID[word]
                Vec.append(Id)
        
        X_Vector.append(Vec)
    
    X_Vector = tf.keras.preprocessing.sequence.pad_sequences(X_Vector, maxlen=90)
    X_Vector = np.array(X_Vector,dtype=np.int32)
    
    return X_Vector

In [ ]:
def PreProcessML(text):
    X.append(text)
    tfidfconverter = TfidfVectorizer(max_features=1000, min_df=5, max_df=0.7, stop_words=stopwords.words('arabic'))
    X_Vec = tfidfconverter.fit_transform(X).toarray()
    
    return X_Vec

In [ ]:
app = Flask(__name__)

@app.route('/test',methods=["POST"])
def Test():
    if request.method == "POST":
        ListOfOutput = ['AE','BH','DZ','EG','IQ','JO','KW','LB','LY','MA','OM','PL','QA','SA','SD','SY','TN','YE']
        req_json = request.json
        test_Sent = req_json["Sent"]
        Input = PreProcess(test_Sent,word2ID)
        InputML = PreProcessML(test_Sent)
        Res = model.predict(Input)
        ResML = modelML.predict([InputML[len(X)-1]])
        Index = Res.argmax()
        IndexML = ResML.argmax()
        Output = ListOfOutput[Index]
        OutputML = ListOfOutput[IndexML]
        
        return jsonify({"response_DL":Output,"response_ML":Output})

In [8]:
if __name__ == '__main__':
    app.run(debug=False,port=9090)